Recarga de Modulos en cada ejecución

In [ ]:
%load_ext autoreload
%autoreload 2

Credenciales de conexión

In [16]:
'''
Lectura de Credenciales en config.json
'''
import json

with open('config.json', 'r') as configfile:
    config = json.load(configfile)
#print(config["sql_server"]["servidor"])


Importación de Módulos
Datos de Entrada

In [17]:
import Modulo.DBConnection as db
from Modulo.StructureMigration import Migration

#Datos de Entrada
origin_database = ''
origin_schema ='python'
origin_table = 'EMPLOYEES'

destiny_database = 'PRUEBA'
destiny_schema = 'oracle'
destiny_table = origin_table

Creación de conexión a Bases de Datos (Origen y Destino)

In [18]:

#Conexión a Base de datos fuente (Origen de Datos)
origin_credentials = config["oracle"]
origin_connection = db.Oracle(origin_credentials["user"], origin_credentials["password"], origin_credentials["server"], origin_credentials["servicename"], origin_credentials["port"])
origin_result = origin_connection.crear_conexion()

#Conexión a Base de datos destino
destiny_credentials = config["sqlserver"]
destiny_connection = db.SQLServer(destiny_credentials["user"], destiny_credentials["password"], destiny_credentials["server"], 'PRUEBA',destiny_credentials["port"])
destiny_result = destiny_connection.crear_conexion()


Creación de Objeto de Migración

In [ ]:

if origin_result == True and destiny_result ==True:
    print('Conexión Realizada con exito a Origen y Destino!')

    #Objeto de Migracion
    migracion = Migration()
    migracion.set_origin(origin_connection,origin_database, origin_schema, origin_table)
    migracion.set_destiny(destiny_connection,destiny_database, destiny_schema, destiny_table)
   
    print(f'La migración es la siguiente: {migracion.migration_type()}')

else:
    print('Error en la conexión:')
    if origin_result == True:
        print('\n', f'Conexión exitosa con el origen: {origin_connection.type}')
    else:
        print('\n', f'Conexión fallida con el origen: {origin_connection.type}')
        print(origin_result)

    if destiny_result == True:
        print('\n', f'Conexión exitosa con el destino: {destiny_connection.type}')
    else:
        print('\n', f'Conexión fallida con el destino: {destiny_connection.type}')
        print(destiny_result)

In [ ]:
#structure = migracion.get_origin_structure()
#structure
query = migracion.generate_destiny_create_table()
print(query)

Ejecución de Migración de Estructura

In [ ]:
ejecucion_migracion = migracion.run_structure_migration()

if ejecucion_migracion == True:
    print('Estructura creada con exito!')
else:
    print(f'Se encontro un error en la ejecución:\n{ejecucion_migracion}')

MIGRACION DE DATOS

In [ ]:
import Modulo.QueryTemplates as qt
import re
origin_data = origin_connection.ejecutar_consulta_json(f'SELECT * FROM {origin_table}')
columns = '\n'.join(migracion.destiny_create_table.splitlines()[11:-3]).replace('NOT NULL', '').replace('NULL', '')
#match = re.search(r'CREATE TABLE .*?\((.*?)\)', migracion.destiny_create_table, re.DOTALL)

insert_query = qt.templates["sqlserver"]["insert_json"].replace(
        '@@JSON@@', origin_data
    ).replace(
        "@@DATABASE@@", destiny_database
    ).replace(
        "@@SCHEMA@@", destiny_schema
    ).replace(
        '@@TABLE@@', destiny_table
    ).replace('@@COLUMNS@@', columns)

print(insert_query)